In [ ]:
import flowkit as fk

from bokeh.io import show
from pathlib import Path

import pandas as pd

def load_sample(subject_id, data_dir):
    data_path = Path(data_dir)
    sample = fk.Sample(
        data_path / "samples" / (subject_id + ".fcs"),
        sample_id=f"patient-{subject_id}"
    )
    # Correct channel idx issues
    if sample.pnn_labels[0]=='FS PEAK':
        sample.fluoro_indices=[3,4,5,6,7,8,9,10,11,12]
        sample.scatter_indices=[1,2]
    else:
        sample.fluoro_indices=[2,3,4,5,6,7,8,9,10,11]
        sample.scatter_indices=[0,1]
    sample.id = subject_id

    detectors = [sample.pnn_labels[i] for i in sample.fluoro_indices]
    fluorochromes = [sample.pns_labels[i] for i in sample.fluoro_indices]
    mat = pd.read_csv(
        data_path / "compensation_matrices" / (subject_id + ".csv"),
        index_col=0
    )
    sample.compensation = fk.Matrix(
        mat.values,
        detectors=detectors,
        fluorochromes=fluorochromes
    )
    sample.apply_compensation(sample.compensation)
    return sample

    
sample = load_sample("35129425", "mock_output")
xform = fk.transforms.AsinhTransform(
    param_t=5, param_m=1, param_a=1
)
sample.apply_transform(xform)
sample.compensation.as_dataframe()

,FL1 INT LOG,FL2 INT LOG,FL3 INT LOG,FL4 INT LOG,FL5 INT LOG,FL6 INT LOG,FL7 INT LOG,FL8 INT LOG,FL9 INT LOG,FL10 INT LOG
FL1 INT LOG,1.000,0.013,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000
FL2 INT LOG,0.176,1.000,0.102,0.000,0.006,0.000,0.000,0.000,0.000,0.000
FL3 INT LOG,0.062,0.391,1.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000
FL4 INT LOG,0.001,0.071,0.307,1.000,0.001,0.001,0.007,0.000,0.000,0.000
FL5 INT LOG,0.000,0.017,0.096,0.504,1.000,0.000,0.000,0.014,0.000,0.000
FL6 INT LOG,0.000,0.000,0.002,0.052,0.000,1.000,0.003,0.099,0.000,0.000
FL7 INT LOG,0.000,0.000,0.002,0.695,0.010,0.514,1.000,0.152,0.000,0.000
FL8 INT LOG,0.000,0.000,0.000,0.158,0.078,0.079,0.000,1.000,0.000,0.000
FL9 INT LOG,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.001,1.000,0.022
FL10 INT LOG,0.069,0.131,0.008,0.001,0.001,0.000,0.000,0.001,0.156,1.000


In [31]:
p = sample.plot_scatter_matrix(source="xform")
show(p)

In [32]:
lmap = {
    key: value for key, value in zip(sample.pns_labels, sample.pnn_labels)
}

In [33]:
sample.pns_labels

['FS INT LIN',
 'SS INT LIN',
 'CD34 FITC',
 'CD13 PE',
 'CD38 ECD',
 'CD7 PC5.5',
 'CD33 PC7',
 'CD56 APC',
 'APC700',
 'CD117 APC750',
 'HLA-DR PB',
 'CD45 KO',
 'TIME']

In [34]:
p = sample.plot_scatter(lmap["CD7 PC5.5"], lmap["CD38 ECD"])
show(p)
p = sample.plot_scatter(
    lmap["CD45 KO"],
    lmap["SS INT LIN"],
    source="xform"
)
show(p)